# Lecture 22 – Case Study

## Data 94, Spring 2021

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

Table.interactive_plots()

Note: We're not going to be able to work through this entire notebook in lecture; you should definitely review whatever we don't get a chance to finish.

## World University Rankings 2020

Our dataset comes from Times Higher Education (THE)'s [World University Rankings 2020](https://www.timeshighereducation.com/world-university-rankings/2020/world-ranking#!/page/0/length/25/sort_by/rank/sort_order/asc/cols/stats). These are slightly outdated as there is a 2021 ranking now, but the data is still relevant.

In [ ]:
world = Table.read_table('data/World_University_Rank_2020.csv')

In [ ]:
world

It's always good to check how many schools we're dealing with:

Some columns (`'Number_students'`, `'International_Students'`, `'Percentage_Female'`, `'Percentage_Male'`) have commas and percentage symbols, meaning they can't be stored as integers. Let's clean them.

In [ ]:
# Notice how we use apply here!
def remove_symbol(s):
    return int(s.replace('%', '').replace(',', ''))

In [ ]:
# Remember, the result of calling apply is an array


In [ ]:
world = world.with_columns(
    'Number_students', world.apply(remove_symbol, 'Number_students'),
    'International_Students', world.apply(remove_symbol, 'International_Students'),
    'Percentage_Female', world.apply(remove_symbol, 'Percentage_Female'),
    'Percentage_Male', world.apply(remove_symbol, 'Percentage_Male')
)

Now we can sort by any numeric column we want.

It seems like the above schools didn't report their sex breakdown, since 0% is the listed percentage of female and male students.

Let's start asking questions.

## Which universities have the most international students?

We have an `'International_Students'` column, but that tells us the **percentage** of international students at each school. Let's update that label to be more clear – let's change the label `'International_Students'` to be `'% International'`.

Then, to compute the **number** of international students at each school, we take the number of students at each school, multiply by the percentage of international students at each school, and divide by 100.

We should probably round the result, since we can't have fractional humans.

In [ ]:
num_international = ...
num_international

We can add this as a column to our table:

In [ ]:
world = world.with_columns(
    '# International', num_international
)

And we can sort by this column, while also selecting a subset of all columns just to focus on what's relevant:

In [ ]:
world.select('University', 'Country', 'Number_students', '% International', '# International') \
     .sort('# International', descending = True)

This tells us that the University of Melbourne has the most international students, with 21,797. That's larger than many universities!

There are no US universities in the top 10 here. How can we find the universities in the US with the most international students?

### [Quick Check 1](https://edstem.org/us/courses/3251/lessons/11661/slides/55875)

Fill in the blanks so that the resulting table contains **the 15 universities in the US with the most international students, sorted by number of international students in decreasing order**.

In [ ]:
# __(a)__ means blank a

# world.select('University', 'Country', 'Number_students', '% International', '# International') \
#      .where(__(a)__, __(b)__) \
#      .sort('# International', __(c)__) \
#      .take(__(d)__)

If you do a quick Google search for "US universities with the most international students", you'll see NYU is usually #1. Cool!

## How do the rankings actually work?

Times Higher Education's [website](https://www.timeshighereducation.com/world-university-rankings/world-university-rankings-2020-methodology) tells us the methodology they use to rank universities:

<img src='https://www.timeshighereducation.com/sites/default/files/wur_2020_methodology_table.jpg' width=500>

This means they come up with a `'Teaching'`, `'Research'`, `'Citations'`, `'International_Outlook'`, and `'Industry_Income'` score from 0 to 100 for each school, then compute a weighted average according to the above percentages to compute a school's `'Score_Result'`, which is how the schools are ranked.

Let's confirm this ourselves. First, let's get a subset of the columns since they won't all be relevant here.

In [ ]:
scores_only = world.select('Score_Rank', 'University', 'Teaching', 'Research', 'Citations', 'International_Outlook', 'Industry_Income', 'Score_Result')

In [ ]:
scores_only

The graphic tells us that the weights for each column are:
- `'Teaching'`: 0.3
- `'Research'`: 0.3
- `'Citations'`: 0.3
- `'International_Outlook'`: 0.075
- `'Industry_Income'`: 0.025

(Remember, to convert from percentage to proportion we divide by 100.)

Let's try and apply this to the school at the very top of the table, University of Oxford.

The result, 95.4175, matches what we see in the `'Score_Result'` column for University of Oxford.

We can apply the above formula to all rows in our table as well.

In [ ]:
score_result_manual_calculation = \
0.3 * scores_only.column('Teaching') + \
0.3 * scores_only.column('Research') + \
0.3 * scores_only.column('Citations') + \
0.075 * scores_only.column('International_Outlook') + \
0.025 * scores_only.column('Industry_Income')

In [ ]:
score_result_manual_calculation

To confirm that the results we got match the `'Score_Result'` column in `scores_only`, we can add the above array to our table:

In [ ]:
scores_only.with_columns(
    'Score Result Manual', score_result_manual_calculation
)

This shows we've successfully reverse-engineered how the rankings work!

## What if we want to change the methodology?

Now that we know how to compute `'Score_Result'`s using THE's percentages, we can also pick our own percentages if we want to prioritize different components in our ranking.

For instance, we may feel like THE's methodology places too much emphasis on research – together, `'Research'` and `'Citations'` make up 60% of the overall score.

We could choose to use the following breakdown, which we'll call "Breakdown 1":
- `'Teaching'`: 60%
- `'International_Outlook'`: 30%
- `'Industry_Income'`: 10%

This gives us new overall scores for each school; we can add this column to our table and sort by it.

In [ ]:
scores_only = scores_only.with_columns(
    'Breakdown 1', breakdown_1
)

In [ ]:
scores_only.sort('Breakdown 1', descending = True)

In [ ]:
scores_only.sort('Breakdown 1', descending = True).take(23)

Note that when we choose this methodology, UC Berkeley is ranked much lower (24th instead of 13th). This is likely due to:
- 1. UC Berkeley's exceptionally high `'Research'` and `'Citations'` scores not being included in the ranking
- 2. UC Berkeley's large class sizes giving it a comparatively low `'Teaching'` score
- 3. UC Berkeley's low score for `'Industry_Income'`. This component factors in the amount that the university receives in funding from industrial partners – given that it's a public school it's unsurprising that this amount is low, but also many "wealthy" universities have a relatively low score here too, so it's not clear how much this should matter (see [here](https://www.timeshighereducation.com/world-university-rankings/world-university-rankings-2020-methodology) for more).

Maybe we want to place some emphasis on research, but not as much as was placed in the initial ranking. We could then make "Breakdown 2":

- `'Teaching'`: 50%
- `'Research'`: 15%
- `'Citations'`: 15%
- `'International_Outlook'`: 15%
- `'Industry_Income'`: 5%

### [Quick Check 2](https://edstem.org/us/courses/3251/lessons/11661/slides/55876)

Assign `breakdown_2` to an array of overall scores for schools calculated according to our Breakdown 2 above, and add it as a column to `scores_only` with the label `'Breakdown 2'`. _Hint: Start by copying our code for `breakdown_1`, which was:_

```py
breakdown_1 = 0.6 * scores_only.column('Teaching') \
            + 0.3 * scores_only.column('International_Outlook') \
            + 0.1 * scores_only.column('Industry_Income')
```

In [ ]:
# Answer QC here before running the next cell

In [ ]:
scores_only.sort('Breakdown 2', descending = True)

"Breakdown 2" is much closer to THE's actual breakdown and the ordering here reflects that.

What do you care about in a university? Try your own breakdown!

We should note though that we haven't really thought about **how** THE comes up with the scores for each of the five categories (or the fact that university rankings have inherent flaws).

## Which countries have the most universities in the ranking?

Back to the full `world` table:

In [ ]:
world

To determine the number of universities per country, we can group by `'Country'`:

It's a good idea to sort too:

How do we get the number of universities in each country with at least 25 universities on the list?

Run the cell below to see a bar graph of the number of universities in each country above. 

In [ ]:
world.group('Country').where('count', are.above_or_equal_to(25)).sort('count').barh('Country')

No surprises here!

We could also determine the average `'Score_Rating'` for every country in the list.

We need to select the relevant columns here and then sort.

This tells us which countries have the "best" universities according to ranking. However, many of the countries at the top are small:

There are only two universities on the list from Singapore, and they're both ranked relatively high.

## What's the best university in each country?

It turns out this can be answered by grouping with a particular `collect` function.

In [ ]:
def first(arr):
    return arr.item(0)

When we group by `'Country'` and use `first` as our `collect `function, we get the first row for each country in the table. Since the `world` table is sorted by ranking to begin with, we don't need to sort before grouping.

In [ ]:
world.group('Country', first)

Let's sort the resulting table by ranking (and also extract a few relevant columns):

In [ ]:
world.group('Country', first) \
.sort('Score_Result first', descending = True) \
.select('Score_Rank first', 'Country', 'University first', 'Score_Result first')

By default, the column we grouped by (so `'Country'`) is the left-most column, but because we selected them in a different order in the last line above they appear in a different order.

What the `'Score_Rank first'` column tells us is how highly-ranked the best university in each country is. It tells us the best university in Canada is 17th best in the world, and the best university in Japan is 33rd best in the world – again, according to THE.

**Question:** What if we wanted the second best, or third best, university in each country?

## In which states are the best US universities located?

So far we've been looking at universities across the world. We may want to zoom in on just universities in the US (and also select just a few relevant columns):

In [ ]:
us_only = world.where('Country', 'United States') \
               .select('University', 'Number_students', 'Score_Result')

In [ ]:
us_only

Right now we don't have any information about where these schools are located within the US.

But we can get that information! Let's refer to Wikipedia's article [List of research universities in the United States](https://en.wikipedia.org/wiki/List_of_research_universities_in_the_United_States) for help. There are two tables there: one for "R1: Doctoral Universities – Very high research activity" and one for "R2: Doctoral Universities – High research activity". For simplicity's sake we'll take just the first table, since the majority of schools in `us_only` will be contained in it.

Run the cell below to load that table in.

_If you're curious as to how we downloaded the information from Wikipedia: We used [this site](https://wikitable2csv.ggor.de), which allows you to specify a Wikipedia URL and gives you a CSV file._

In [ ]:
us_r1 = Table.read_table('data/us-r1-universities.csv')
us_r1

Now, let's join the two tables together. We want to look for matches between the `'University'` column in `us_only` and the `'Institution'` column in `us_r1`.

In [ ]:
# Think about why these are the arguments to join!
us_with_state = ...
us_with_state

The above table only has 96 rows, whereas the bar chart earlier told us there were 172 US schools in our rankings table. That likely means we lost rows as a result of:
- School names not matching between the two tables (maybe one column had a typo or alternative name)
- School names not being in `us_r1`

At a glance though it doesn't seem like there are major omissions, since all of the top 10 US schools are still there:

Let's proceed. Since our original goal was to determine the states in which the best schools were, we can group by `'State'` – first with no aggregation function, and then with `np.mean` to look at the average `'Score_Result'` for each state.

In [ ]:
# sort(1) means sort by the column at index 1, which is 'Score_Result mean'


The above tells us that California had the most high-ranked schools in the dataset, while Maryland has the highest average ranking amongst all states in the dataset. However, just like with Singapore earlier, there aren't many schools from Maryland:

## Followup – how do public and private schools in the US compare?

The `us_r1` table brought us more information that we haven't yet used – it gave us information about whether each school is public or private! Let's do some initial queries.

In [ ]:
us_with_state.group('Control')

In [ ]:
us_with_state.group('Control', np.mean)

It seems like there are fewer US private schools in the dataset, but they're ranked higher than the public schools in the dataset on average. They're also smaller on average, at ~16,000 to ~31,000.

Since we have two categories, we can group:

In [ ]:
us_with_state.group(['State', 'Control'])

But an easier way to look at the above would be to create a pivot table:

If we want the states with the most public schools in the ranking:

What if we want the average `'Score_Result'` for each state, separated by public vs. private? We can do that too.

This, in theory, gives us the states with the "best" public universities at the top.

This doesn't mean a whole lot, since the number of public universities in each state is wildly different.

For example, we see Minnesota (`'MN'`) is the second row – there's only one school in the dataset from Minnesota!

## A warning

If you pivot with columns that don't really make sense, you'll get weird results:

In [ ]:
us_with_state.pivot('State', 'University')

## Done!

Make sure to review all of the code in this notebook, as it's all great review for the quiz.